In [ ]:
#this dataset can be used in future (it is used as reference in the current model)
import tensorflow_datasets as tfds
import pandas as pd
splits = ['train', 'test']
dataset = tfds.load('math_qa', split=splits)

In [ ]:
train_df=pd.DataFrame(dataset[0])
test_df=pd.DataFrame(dataset[1])

In [ ]:
train_df.shape,test_df.shape
train_df.head()

,Problem,Rationale,annotated_formula,category,correct,correct_option,linear_formula,options
0,tf.Tensor(b'pascal has 96 miles remaining to c...,"tf.Tensor(b'""let the current speed be x miles ...","tf.Tensor(b'divide(add(divide(96, 16), sqrt(ad...","tf.Tensor(b'physics', shape=(), dtype=string)","tf.Tensor(b'b', shape=(), dtype=string)","tf.Tensor(b'8', shape=(), dtype=string)","tf.Tensor(b'divide(n0,n2)|divide(n3,const_100)...","tf.Tensor(b'a ) 6 , b ) 8 , c ) 10 , d ) 12 , ..."
1,tf.Tensor(b'a library has an average of 510 vi...,"tf.Tensor(b'""since the month begins with a sun...",tf.Tensor(b'divide(add(multiply(add(floor(divi...,"tf.Tensor(b'general', shape=(), dtype=string)","tf.Tensor(b'a', shape=(), dtype=string)","tf.Tensor(b'285', shape=(), dtype=string)","tf.Tensor(b'add(const_3,const_4)|divide(n2,#0)...","tf.Tensor(b'a ) 285 , b ) 227 , c ) 271 , d ) ..."
2,tf.Tensor(b'what is the volume of an tetrahedr...,tf.Tensor(b'the answer is sqr ( 2 ) / 12 = 0.1...,"tf.Tensor(b'divide(sqrt(const_2), add(const_10...","tf.Tensor(b'geometry', shape=(), dtype=string)","tf.Tensor(b'c', shape=(), dtype=string)","tf.Tensor(b'0.11785113', shape=(), dtype=string)","tf.Tensor(b'add(const_10,const_2)|sqrt(const_2...","tf.Tensor(b""['a ) 3.11545466', 'b ) 0.76678113..."
3,tf.Tensor(b'a multiple choice test consists of...,"tf.Tensor(b'""5 choices for each of the 4 quest...","tf.Tensor(b'power(5, 4)', shape=(), dtype=string)","tf.Tensor(b'general', shape=(), dtype=string)","tf.Tensor(b'c', shape=(), dtype=string)","tf.Tensor(b'625', shape=(), dtype=string)","tf.Tensor(b'power(n1,n0)|', shape=(), dtype=st...","tf.Tensor(b'a ) 24 , b ) 120 , c ) 625 , d ) 7..."
4,tf.Tensor(b'in the first 10 overs of a cricket...,"tf.Tensor(b'""required run rate = [ 292 - ( 3.2...","tf.Tensor(b'divide(subtract(292, multiply(10, ...","tf.Tensor(b'gain', shape=(), dtype=string)","tf.Tensor(b'a', shape=(), dtype=string)","tf.Tensor(b'6.5', shape=(), dtype=string)","tf.Tensor(b'multiply(n0,n1)|subtract(n3,#0)|di...","tf.Tensor(b'a ) 6.5 , b ) 6.22 , c ) 6.29 , d ..."


In [ ]:
#Installing the packages for Gemini which is used in question generation
!pip install -q -U google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.8/146.8 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.5/664.5 kB 9.2 MB/s eta 0:00:00


In [ ]:
#Run the cells to work with Gemini API
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
# Used to securely store your API key
from google.colab import userdata

In [ ]:
#import your API key else it won't run
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


In [ ]:
model = genai.GenerativeModel('gemini-pro')

In [ ]:
#prompt is given to interact with the model
%%time
category = "geometry"  # this category will be given by the model
prompt=f"Generate a hard level question from {category} which has a numerical or few words answer"
response = model.generate_content(prompt)

CPU times: user 64 ms, sys: 10.2 ms, total: 74.2 ms
Wall time: 3.71 s


In [ ]:
#example question
to_markdown(response.text)

> In a right triangle with legs of length 3 and 4, what is the area of the circle inscribed in the triangle?

In [ ]:
import numpy as np

# Convert TensorFlow tensors to NumPy arrays
category_array = train_df['category'].apply(lambda x: x.numpy())

# Get unique categories
unique_categories = np.unique(category_array)
print(unique_categories)



[b'gain' b'general' b'geometry' b'other' b'physics' b'probability']


In [ ]:
import numpy as np
import pandas as pd

# Define the list of categories
categories = ['gain', 'general', 'geometry', 'other', 'physics', 'probability']

# Define the user ID
user_id = 1

# Initialize empty lists for each category
category_data = {category: {'total_correct': np.random.randint(0, 10),
                            'num_questions': np.random.randint(0, 20),
                            'last_time_asked': np.random.randint(0, 20)} for category in categories}

# Create a DataFrame for each category
category_dfs = []
for category, data in category_data.items():
    df = pd.DataFrame({'User_ID': [user_id], **data})
    df['Category'] = category
    category_dfs.append(df)

# Concatenate DataFrames along the columns axis
user_df = pd.concat(category_dfs, ignore_index=True)

# Print the generated dataset
print(user_df)


   User_ID  total_correct  num_questions  last_time_asked     Category
0        1              6              3               16         gain
1        1              1             10               18      general
2        1              1              3                0     geometry
3        1              5             13                2        other
4        1              9             10                9      physics
5        1              7             13               14  probability


In [8]:
import numpy as np
import pandas as pd

def generate_user_data(user_id):
    # Define the list of categories
    categories = ['gain', 'general', 'geometry', 'other', 'physics', 'probability']

    # Generate random values for total_correct, num_questions, and last_time_asked for each category
    total_correct = np.random.randint(0, 10, size=len(categories))
    num_questions = np.random.randint(0, 20, size=len(categories))
    last_time_asked = np.random.randint(0, 100, size=len(categories))

    # Create a list of dictionaries for each category
    category_data = []
    for i, category in enumerate(categories):
        category_dict = {
            'User_ID': user_id,
            'Category': category,
            'total_correct': total_correct[i],
            'num_questions': num_questions[i],
            'last_time_asked': last_time_asked[i]
        }
        category_data.append(category_dict)

    # Create a DataFrame for the user data
    user_df = pd.DataFrame(category_data)

    return user_df

# Example usage:
user_id = 1
user_data = generate_user_data(user_id)
print(user_data)


   User_ID     Category  total_correct  num_questions  last_time_asked
0        1         gain              7              5               43
1        1      general              1             17               98
2        1     geometry              7             18               19
3        1        other              8             16               75
4        1      physics              6             11               75
5        1  probability              8             17               12


In [ ]:
!python -m pip install pyqlearning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyqlearning: filename=pyqlearning-1.2.7-py3-none-any.whl size=54949 sha256=2f511668d248a9508aadfea3091adfe08b16cd505aab0e030676513d94c1025a
  Stored in directory: /root/.cache/pip/wheels/ac/74/f6/ce67a3cdedf4b3d14351f1449a8edd8dafa0b8c98aac9a6eec
Successfully built pyqlearning


In [12]:
#reinforcement learning model to maximise the score
import numpy as np

class QLearningTopicSelector:
    def __init__(self, n_topics, learning_rate=0.1, discount_factor=0.9, epsilon=0.1):
        self.n_topics = n_topics
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.epsilon = epsilon

        self.q_table = np.zeros((n_topics,))
        self.state = np.zeros((n_topics, 3))  # total correct, total questions, last question asked

    def choose_topic(self):
        if np.random.uniform(0, 1) < self.epsilon:
            return np.random.choice(self.n_topics)
        else:
            return np.argmax(self.q_table)

    def predict(self):
        return np.argmax(self.q_table)

    def update_q_table(self, topic, reward):
        current_q = self.q_table[topic]
        new_q = current_q + self.learning_rate * (reward - current_q)
        self.q_table[topic] = new_q

    def get_q_table(self):
        return self.q_table

    def get_state(self):
        return self.state

    def set_state(self, state):
        self.state = state

def run_episodes(topic_selector, num_episodes=1000):
    for episode in range(num_episodes):
        topic = topic_selector.choose_topic()
        reward = calculate_reward(topic_selector.get_state(), topic)
        topic_selector.update_q_table(topic, reward)

def calculate_reward(state, topic):
    # Calculate reward based on the performance of all topics
    correct_percentages = []
    incorrect_percentages = []
    for i in range(len(state)):
        if topic == i:
            if state[i][1] > 0:
                correct_percentages.append((state[i][0] + 1) / (state[i][1] + 1))
                incorrect_percentages.append((state[i][0] - 1) / (state[i][1] + 1))
            else:
                correct_percentages.append(1)
                incorrect_percentages.append(0)
        else:
            if state[i][1] > 0:
                correct_percentages.append(state[i][0] / state[i][1])
                incorrect_percentages.append(state[i][0] / state[i][1])
            else:
                correct_percentages.append(0)
                incorrect_percentages.append(0)
    s = 0
    ratio = 0.8  # this is the assumed ratio of the correctness and it should be on the model to learn
    weight = 0.02
    for i in range(len(state)):
        s += correct_percentages[i] * ratio + incorrect_percentages[i] * (1 - ratio)  # assuming 80-20 ratio
    return s / len(state) + weight * state[topic][2]

def calculate_last_time_asked(num_questions, last_time_asked):
    # Assuming a linear increase in last_time_asked based on the number of questions asked
    return last_time_asked + num_questions

def calculate_next_time_to_ask(last_time_asked, interval):
    return last_time_asked + interval

# Example usage:

a = np.array([
    [3, 6, 4], #gain
    [1, 4, 8], #general
    [5, 5, 10], #geometry
    [1, 1, 12], #other
    [2,10,8],   #physics
    [9,10,10]  #probability
])

n_topics = len(a)  # Assuming you have 6 topics
last_time_asked = np.max(a[:, 2])  # Get the last time the same question was asked
interval = 18  # Assuming the interval after which the same question is asked again
next_time_to_ask = calculate_next_time_to_ask(last_time_asked, interval)

topic_selector = QLearningTopicSelector(n_topics)
topic_selector.set_state(a)

# Run episodes to train the topic selector
run_episodes(topic_selector)
predicted_topic = topic_selector.predict()
if predicted_topic==1:{
    print("gain")
}
if predicted_topic==2:{
    print("general")
}
if predicted_topic==3:{
    print("geometry")
}
if predicted_topic==4:{
    print("other")
}
if predicted_topic==5:{
    print("physics")
}
if predicted_topic==6:{
    print("probability")
}
#note:the categories question are updating every time you run this cell , hence you see different outputs
#Output:physics - this shows that the user i performing poorly in physics category and the question should be asked from it
#in future more parameters like speed can also be added in the numpy array as the fourth parameter

physics


In [1]:
def user_analytics(user_df):
  total_correct =0
  num_questions = 0
  for i in range(6):
    total_correct+=user_df.iloc[i,:]['total_correct']
    num_questions+=user_df.iloc[i,:]['num_questions']
  return (total_correct,num_questions)

In [2]:
def user_extract(user_df):
  data=[]
  for i in range(6):
    category=[user_df.iloc[i,:]['total_correct'],user_df.iloc[i,:]['num_questions'],user_df.iloc[i,:]['last_time_asked']]
    data.append(category)
  return np.array(data)


In [3]:
def user_performance(user_df):
  total_correct,num_questions = user_analytics(user_df)
  return 100*total_correct/num_questions

In [4]:
def max_performance(user_df,questions):
  total_correct,num_questions=user_analytics(user_df)
  return 100*(total_correct+questions)/(num_questions+questions)

In [5]:
def updating_user(user_df,index,correct):
  if(correct):
    user_df.loc[index,'total_correct']+=1
  for i in range(6):
    user_df.loc[i,'last_time_asked']+=1

  user_df.loc[index,'last_time_asked']=0
  user_df.loc[index,'num_questions']+=1

In [6]:
def exam_simulation(users,questions,ratio=0.8):
  s=0
  for user in users:
    initial=user_performance(user)
    limit=max_performance(user,questions)
    for q in range(questions):
      topic_selector.set_state(user_extract(user))
      chosen=topic_selector.predict()

      if random.random() < ratio: #80% times user answers correctly
        updating_user(user,chosen,True)
      else:
        updating_user(user,chosen,False)
    final=user_performance(user)


    s+=100*(final-initial)/(limit-initial)
  return s/len(users);

In [9]:
users_list_1=[]
for i in range(100):
  a_user_data=generate_user_data(i)
  users_list_1.append(a_user_data)
users_list_2=[]
for i in range(100):
  a_user_data=generate_user_data(i)
  users_list_2.append(a_user_data)
users_list_3=[]
for i in range(100):
  a_user_data=generate_user_data(i)
  users_list_3.append(a_user_data)

In [16]:
import random
print("Percentage improvement for 100 users over a 5 question exam :",exam_simulation(users_list_1,5))
print("Percentage improvement for 100 users over a 10 question exam :",exam_simulation(users_list_2,10))
print("Percentage improvement for 100 users over a 20 question exam :",exam_simulation(users_list_2,20))

Percentage improvement for 100 users over a 5 question exam : 69.15186729610794
Percentage improvement for 100 users over a 10 question exam : 74.14380002080816
Percentage improvement for 100 users over a 20 question exam : 81.98053339395996


In [17]:
a = np.array([
    [3, 6, 4], #gain
    [1, 4, 8], #general
    [51, 5, 10], #geometry
    [1, 1, 12], #other
    [9,10,8],   #physics
    [9,10,10]  #probability
])
topic_selector.set_state(a)
topic_selector.predict()
#accuracy

5